In [118]:
import pandas as pd
import numpy as np

df = pd.read_csv('data.csv', parse_dates=['일자'], encoding= 'cp949')

In [119]:
df.head()

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,2024-02-21,407000,3000,0.74,402500,408000,398000,153418,61943906000,95238000000000,234000000
1,2024-02-20,404000,6500,1.64,397500,404500,392000,157923,63104987000,94536000000000,234000000
2,2024-02-19,397500,-12500,-3.05,412000,412500,395500,289496,115932894000,93015000000000,234000000
3,2024-02-16,410000,14500,3.67,401500,412000,401000,354959,144830615000,95940000000000,234000000
4,2024-02-15,395500,7500,1.93,393500,397500,389000,192353,75698271500,92547000000000,234000000


In [120]:
type(df['일자'].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [121]:
df = df[['일자', '시가', '저가', '고가', '종가']].copy()
df['year'] = df['일자'].dt.year #연도 칼럼
df['month'] = df['일자'].dt.month #월 칼럼 새롭게 데이터 프레임에 추가함
df.head()

,일자,시가,저가,고가,종가,year,month
0,2024-02-21,402500,398000,408000,407000,2024,2
1,2024-02-20,397500,392000,404500,404000,2024,2
2,2024-02-19,412000,395500,412500,397500,2024,2
3,2024-02-16,401500,401000,412000,410000,2024,2
4,2024-02-15,393500,389000,397500,395500,2024,2


In [122]:
gb = df.groupby(['year', 'month'])
gb.get_group((2024,2)).head()

,일자,시가,저가,고가,종가,year,month
0,2024-02-21,402500,398000,408000,407000,2024,2
1,2024-02-20,397500,392000,404500,404000,2024,2
2,2024-02-19,412000,395500,412500,397500,2024,2
3,2024-02-16,401500,401000,412000,410000,2024,2
4,2024-02-15,393500,389000,397500,395500,2024,2


In [123]:
mapping = { '시가' : 'first', '저가' : min, '고가' : max, '종가' : 'last' }
gb.agg(mapping)

시가      저가      고가      종가
year month                                
2023 8      554000  522000  558000  526000
     9      469500  461500  544000  522000
     10     404500  382500  501000  456000
     11     444000  375500  500000  377500
     12     421500  409500  448000  428500
2024 1      379000  362000  429500  429500
     2      402500  373000  412500  381500

In [124]:
df.groupby(pd.Grouper(key='일자', freq='m')).agg(mapping) #grouper 함수를 활용해서 groupby 실행 규칙 정의

,시가,저가,고가,종가
일자,,,,
2023-08-31,530000,522000,558000,544000
2023-09-30,544000,461500,544000,476500
2023-10-31,467000,382500,501000,385500
2023-11-30,389500,375500,500000,454500
2023-12-31,444000,409500,448000,427500
2024-01-31,425000,362000,429500,381000
2024-02-29,378000,373000,412500,407000


In [125]:
df2 = pd.read_csv('data.csv', parse_dates=['일자'], encoding= 'cp949')
df2['전일거래량'] = df2['거래량'].shift(1)
df2[['거래량', '전일거래량']]

,거래량,전일거래량
0,153418,NaN
1,157923,153418.0
2,289496,157923.0
3,354959,289496.0
4,192353,354959.0
...,...,...
118,249493,246291.0
119,191114,249493.0
120,191350,191114.0
121,195045,191350.0


In [126]:
increase = df2['거래량'] > df2['전일거래량']
df2[increase].value_counts()

일자          종가      대비      등락률     시가      고가      저가      거래량      거래대금          시가총액             상장주식수      전일거래량   
2023-08-21  526000  -3000   -0.57   530000  534000  522000  251626   132693989000  123084000000000  234000000  195045.0    1
2023-11-15  451500   14500   3.32   460000  460000  446000  354065   160238622500  105651000000000  234000000  186484.0    1
2023-11-21  447000   3000    0.68   450000  452000  444000  158805   71121360000   104598000000000  234000000  136059.0    1
2023-11-23  448500   6500    1.47   440500  450500  440500  142291   63630154000   104949000000000  234000000  134085.0    1
2023-11-27  434500  -8000   -1.81   446000  447500  432500  154442   67655960000   101673000000000  234000000  140561.0    1
                                                                                                                          ..
2023-11-02  391500   14000   3.71   383500  394000  380000  485126   188465974500  91611000000000   234000000  442376.0    1
2023-

- 간단한 모멘텀 투자 전략 구현

In [127]:
yeild = df['종가'] / df['종가'].shift(7)
increase = yeild >= 1.03
len(df[increase])

43

In [128]:
df['ma3'] = df['종가'].rolling(3).mean() 
df.head() #rolling 함수를 활용해서 이동 평균을 구한 것

,일자,시가,저가,고가,종가,year,month,ma3
0,2024-02-21,402500,398000,408000,407000,2024,2,NaN
1,2024-02-20,397500,392000,404500,404000,2024,2,NaN
2,2024-02-19,412000,395500,412500,397500,2024,2,402833.333333
3,2024-02-16,401500,401000,412000,410000,2024,2,403833.333333
4,2024-02-15,393500,389000,397500,395500,2024,2,401000.000000


In [129]:
#시가가 6일 이동평균선을 돌파하는 경우 상승 추세라고 가정
df['ma6'] = df['종가'].rolling(6).mean().shift(1)
df['cond'] = df['ma6'] < df['시가']
df['cond'].value_counts()

True     74
False    49
Name: cond, dtype: int64

In [130]:
df['pct'] = df['종가'].pct_change(periods=2) #2일간 보유 수익률 계산

In [131]:
df['pct'].head(10)

0         NaN
1         NaN
2   -0.023342
3    0.014851
4   -0.005031
5   -0.053659
6   -0.005057
7    0.015464
8   -0.019060
9   -0.043147
Name: pct, dtype: float64

In [132]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=df['일자'], y=df['pct'], mode='lines', name='주가 5일간 수익률'
    )
)

fig.show()

In [133]:
df_quarter = df[['종가', '시가']].copy()
df_quarter.set_index(df['일자'], inplace=True)
df_quarter.head()

,종가,시가
일자,,
2024-02-21,407000,402500
2024-02-20,404000,397500
2024-02-19,397500,412000
2024-02-16,410000,401500
2024-02-15,395500,393500


In [134]:
df_quarter = df_quarter['시가'].groupby(pd.Grouper(freq='q')).first().to_frame()

In [135]:
df_quarter['quarter'] = df_quarter.index.quarter
df.index = pd.to_datetime(df.index)
df['quarter'] = df.index.quarter
date_strings = df.index
dates = pd.to_datetime(date_strings, format='%Y-%m-%d', errors='coerce')

In [136]:
df_quarter.head()

,시가,quarter
일자,,
2023-09-30,530000,3
2023-12-31,467000,4
2024-03-31,425000,1


In [150]:
fig = go.Figure(data=[go.Candlestick(x=df['일자'],
                                     open=df['시가'],
                                     high=df['고가'],
                                     low=df['저가'],
                                     close=df['종가'])])

fig.add_trace(go.Scatter(x=df['일자'], y=df['ma6'], mode='lines', name='MA(6))'))
fig.update_layout(title='캔들 차트와 이동평균선', xaxis_title='날짜', yaxis_title='가격')
fig.show()